## Download the data

In [1]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]
###### Do not modify here ###### 


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Get the data shape

In [2]:
print('Train', mnist.train.num_examples, 
      'Validation', mnist.validation.num_examples,
      'Test', mnist.test.num_examples)

print('Train images :', X_train1.shape,
      'Labels :', y_train1.shape)

print('Validation images :', X_valid1.shape,
      'Labels :', y_valid1.shape)

print('Test images :', X_test1.shape,
      'Labels :', y_test1.shape)


Train 55000 Validation 5000 Test 10000
Train images : (28038, 784) Labels : (28038,)
Validation images : (2558, 784) Labels : (2558,)
Test images : (5139, 784) Labels : (5139,)


## One-hot encoded y values

In [12]:
y_train_one_hot = np.zeros((len(y_train1), 5))
y_train_one_hot[np.arange(len(y_train1)), y_train1] = 1

y_valid_one_hot = np.zeros((len(y_valid1), 5))
y_valid_one_hot[np.arange(len(y_valid1)), y_valid1] = 1

y_test_one_hot = np.zeros((len(y_test1), 5))
y_test_one_hot[np.arange(len(y_test1)), y_test1] = 1

print('Check some examples of one-hot encoding:')
for i in range(10):
    print(y_train1[i], y_train_one_hot[i], '\n')

print ("Train Label: ", y_train_one_hot.shape)
print ("Validation Label: ", y_valid_one_hot.shape)
print ("Test Label: ", y_test_one_hot.shape)


Check some examples of one-hot encoding:
3 [ 0.  0.  0.  1.  0.] 

4 [ 0.  0.  0.  0.  1.] 

1 [ 0.  1.  0.  0.  0.] 

1 [ 0.  1.  0.  0.  0.] 

0 [ 1.  0.  0.  0.  0.] 

0 [ 1.  0.  0.  0.  0.] 

3 [ 0.  0.  0.  1.  0.] 

1 [ 0.  1.  0.  0.  0.] 

2 [ 0.  0.  1.  0.  0.] 

0 [ 1.  0.  0.  0.  0.] 

Train Label:  (28038, 5)
Validation Label:  (2558, 5)
Test Label:  (5139, 5)


## Construct the layer

In [4]:
def layer(output_dim, input_dim, inputs, layer, activation=None):
#     W = tf.Variable(tf.random_normal([input_dim, output_dim]))
#     b = tf.get_variable(tf.random_normal([1, output_dim]))
#     W = tf.Variable(tf.truncated_normal(shape = (input_dim, output_dim),mean = 0, stddev = 0.1))
#     b = tf.Variable(tf.zeros(output_dim))

    W = tf.get_variable("W" + layer, shape=[input_dim, output_dim], initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable("b" + layer, shape=[1, output_dim], initializer=tf.contrib.layers.xavier_initializer())

    XWb = tf.matmul(inputs, W) + b
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
    return outputs


### contruct the input layer

In [5]:
x = tf.placeholder("float", [None, 784])


### contruct the hidden layers

In [6]:
h1 = layer(output_dim=128, input_dim=784, inputs=x, layer="h1", activation=tf.nn.elu)
h2 = layer(output_dim=128, input_dim=128, inputs=h1, layer="h2", activation=tf.nn.elu)
h3 = layer(output_dim=128, input_dim=128, inputs=h2, layer="h3", activation=tf.nn.elu)
h4 = layer(output_dim=128, input_dim=128, inputs=h3, layer="h4", activation=tf.nn.elu)
h5 = layer(output_dim=128, input_dim=128, inputs=h4, layer="h5", activation=tf.nn.elu)


### contruct the output layer

In [7]:
y_predict = layer(output_dim=5, input_dim=128, inputs=h5, layer="output", activation=None)
y_label = tf.placeholder("float", [None, 5])


### start training

In [24]:
# hyper parameters
lr = 0.005
batch_size = 256
epochs = 350
saturate_limit = 20  # for applying early stopping

# parameters
iterations = int(X_train1.shape[0] / batch_size)
saturate_count = 0
best_acc = 0.
best_epoch = -1

# set the loss function; tf.nn.sparse_softmax_cross_entropy_with_logits required by the homework spec
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_predict, labels=y_label))

# set the optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr) \
                    .minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(y_label, 1), tf.argmax(y_predict, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs):
        for i in range(iterations):
            if(i==0):
                batch_x = X_train1[:batch_size]
                batch_y = y_train_one_hot[:batch_size]
            else:
                batch_x = X_train1[i * batch_size : (i + 1) * batch_size]
                batch_y = y_train_one_hot[i * batch_size : (i + 1) * batch_size]

            sess.run(optimizer, feed_dict={x: batch_x, y_label: batch_y})
        
        # validate
        loss, acc = sess.run([loss_function, accuracy], feed_dict={x: X_valid1, y_label: y_valid_one_hot})
        
        print('-' * 40)
        print('Epoch:', e)
        print('Loss:', loss)
        print('Accurancy:', acc)
        
        if best_acc < acc:
            print('best_acc occurs!')
            best_acc = acc
            best_epoch = e
            saturate_count = 0
        else:
            saturate_count += 1
            print('increment saturate_count:', saturate_count)
            print('current best_acc:', best_acc)
            
            if saturate_count >= saturate_limit:  # stop if saturate
                print('SATURATE!')
                break
                
        print('-' * 40, '\n')
        
        
    print('*' * 60)
    print('Best epoch:', best_epoch)
    print('Best accurancy:', best_acc)
    print('*' * 60, '\n')
    

----------------------------------------
Epoch: 0
Loss: 0.0827843
Accurancy: 0.975371
best_acc occurs!
---------------------------------------- 

----------------------------------------
Epoch: 1
Loss: 0.057227
Accurancy: 0.980844
best_acc occurs!
---------------------------------------- 

----------------------------------------
Epoch: 2
Loss: 0.0718664
Accurancy: 0.981235
best_acc occurs!
---------------------------------------- 

----------------------------------------
Epoch: 3
Loss: 0.0835222
Accurancy: 0.980844
increment saturate_count: 1
current best_acc: 0.981235
---------------------------------------- 

----------------------------------------
Epoch: 4
Loss: 0.0519918
Accurancy: 0.984754
best_acc occurs!
---------------------------------------- 

----------------------------------------
Epoch: 5
Loss: 0.0587146
Accurancy: 0.982408
increment saturate_count: 1
current best_acc: 0.984754
---------------------------------------- 

----------------------------------------
Epoch: 6

----------------------------------------
Epoch: 47
Loss: 0.061274
Accurancy: 0.993745
increment saturate_count: 11
current best_acc: 0.995309
---------------------------------------- 

----------------------------------------
Epoch: 48
Loss: 0.0623124
Accurancy: 0.993745
increment saturate_count: 12
current best_acc: 0.995309
---------------------------------------- 

----------------------------------------
Epoch: 49
Loss: 0.0632081
Accurancy: 0.993745
increment saturate_count: 13
current best_acc: 0.995309
---------------------------------------- 

----------------------------------------
Epoch: 50
Loss: 0.0640072
Accurancy: 0.993745
increment saturate_count: 14
current best_acc: 0.995309
---------------------------------------- 

----------------------------------------
Epoch: 51
Loss: 0.0647343
Accurancy: 0.993745
increment saturate_count: 15
current best_acc: 0.995309
---------------------------------------- 

----------------------------------------
Epoch: 52
Loss: 0.065405
Accur